# Variational Monte Carlo Demonstration
Authors: Hannah Baek, Tobias Tian

## About this Notebook
- In this notebook, we are going to demonstrate how we can use VMC to estimate ground-state energy of a many-body physical system.
- For demonstrating the plausibility of this algorithm, while maintaining simplicity, we decide to follow a classic example in showing the effectiveness of VMC, the helium atom.
- Without highlighting too much of the physical details which can overshadow the idea of VMC, we will restrict our attention to the electronic degrees of freedom of the helium atom. The nucleus, consisting of 2 protons and 2 neutrons, is treated as fixed within the Born-Oppenheimer approximation. This way, the system can be regarded as consisting 2 interacting electrons in the Coulomb potential of a single immobile nucleus. Introducing the nucleus into the system will drastically complicate this demonstration and shift the focus to physics instead of the method itself, as we need to consider the interaction within the nucleus.

## Background Information
- Our objective in this demonstration is to determine the electronic ground-state energy of the helium atom.
- We will be using a trial wavefunction that is dependent on the Cartesian co-ordinates of the two electrons $\mathbf{r}_1, \mathbf{r}_2$: $$\Psi_T(\mathbf{r}_1, \mathbf{r}_2)$$ and we call $\mathbf{R} = (\mathbf{r}_1, \mathbf{r}_2)$ to be a "configuration"
- The ground-state energy is approximated by: $$E_0 \approx \frac{\langle \Psi_T | \hat{H} | \Psi_T \rangle}{\langle \Psi_T | \Psi_T \rangle}$$
- However, the above approximation does not progress naturally, this is where we introduce VMC. We will be sampling according to the trial wavefunction $\Psi_T$ and obtain a list of configurations $[\mathbf{R}_1, \mathbf{R}_2, \dots, \mathbf{R}_n]$. For each of the configuration, we will calculate its local energy: $$E_{\text{loc}, i} = \frac{\hat{H}\Psi_T(\mathbf{R}_i)}{\Psi_T(\mathbf{R}_i)}$$ and take the average to approximate $$E_0 \approx \frac{1}{n}\sum_{i = 1}^{n} E_{\text{loc}, i}$$
- The Hamiltonian, in this helium atom example, $\hat{H}$, is given to be: $$\hat{H} = -\frac{1}{2}\nabla_1^2 - \frac{1}{2}\nabla_2^2 - \frac{2}{r_1} - \frac{2}{r_2} + \frac{1}{r_{12}}$$ where this is true in "atomic units".
- In "atomic units" system, we will be "defining" the constants of $\hbar$ (Planck's constant), $m_e$ (mass of an electron), $e$ (the charge of one electron), $4\pi\varepsilon_0$ (permittivity) to be $1$. The result of such "definitions" is that, length will now be measured in Bohr radii ($a_0$), and energy will now be measured in Hartree ($E_h$).

## Coding Demonstration

### Setup

In [1]:
# Setup, relevant parameters

import numpy as np 

Z = 2.0
a_cusp = 0.5
rng = np.random.default_rng(seed=123)

### Trial Wavefunction Selection

In [2]:
# This function takes in relevant parameters, and a configuration,
# and outputs the calculated value of the wavefunction

def norm(v):
    return np.sqrt(np.dot(v, v))

def distances(R):
    """Returns r1, r2, r12."""
    r1 = norm(R[0])
    r2 = norm(R[1])
    r12 = norm(R[0] - R[1])
    return r1, r2, r12

def log_slater(R):
    """Logarithm of Φ = exp[-Z(r1+r2)]"""
    r1, r2, _ = distances(R)
    return -Z * (r1 + r2)

def log_jastrow(R, beta):
    """Logarithm of Jastrow = u(r12)"""
    _, _, r12 = distances(R)
    if r12 == 0:
        return 0.0
    return a_cusp * r12 / (1 + beta * r12)

def log_psi(R, beta):
    """Total log Ψ_T"""
    return log_slater(R) + log_jastrow(R, beta)

def psi_sq(R, beta):
    """|Ψ_T|^2"""
    return np.exp(2 * log_psi(R, beta))

### Sampling the Trial Wavefunction (Metropolis Algorithm)

In [3]:
# This function takes in the trial wavefunction selected,
# and outputs a list of "accepted" configurations

def initial_R():
    return np.array([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0]], dtype=float)

def metropolis(beta, n_steps, step_size=0.4):
    R = initial_R()
    psi2_old = psi_sq(R, beta)

    samples = [R.copy()]

    for step in range(1, n_steps):

        # Uniform random displacement in [-step_size/2, +step_size/2]
        dR = rng.uniform(
            low=-step_size/2,
            high= step_size/2,
            size=R.shape
        )

        R_new = R + dR
        psi2_new = psi_sq(R_new, beta)

        A = psi2_new / psi2_old

        if A >= 1 or rng.random() < A:
            R = R_new
            psi2_old = psi2_new

        samples.append(R.copy())

    return samples


### Calculating Local Energy

In [4]:
# This function takes in a configuration from the list of samples obtained,
# and outputs the local energy at that configuration

def grad_log_psi(R, beta):
    r1_vec, r2_vec = R
    s = r1_vec - r2_vec

    r1 = norm(r1_vec)
    r2 = norm(r2_vec)
    r12 = norm(s)

    r1_hat = r1_vec/r1 if r1 > 0 else np.zeros(3)
    r2_hat = r2_vec/r2 if r2 > 0 else np.zeros(3)
    s_hat  = s/r12 if r12 > 0 else np.zeros(3)

    g1 = -Z * r1_hat
    g2 = -Z * r2_hat

    if r12 > 0:
        up = a_cusp / (1 + beta*r12)**2
        g1 += up * s_hat
        g2 -= up * s_hat

    return g1, g2

def laplacian_log_psi(R, beta):
    r1_vec, r2_vec = R
    s = r1_vec - r2_vec

    r1 = norm(r1_vec)
    r2 = norm(r2_vec)
    r12 = norm(s)

    lap1 = -2*Z/r1 if r1 > 0 else 0
    lap2 = -2*Z/r2 if r2 > 0 else 0

    if r12 > 0:
        up = a_cusp / (1 + beta*r12)**2
        upp = -2*a_cusp*beta/(1 + beta*r12)**3
        lap_u = upp + (2/r12)*up
        lap1 += lap_u
        lap2 += lap_u

    return lap1 + lap2

def energy_local(R, beta):
    r1, r2, r12 = distances(R)
    g1, g2 = grad_log_psi(R, beta)
    lap = laplacian_log_psi(R, beta)

    kinetic = -0.5 * (lap + np.dot(g1, g1) + np.dot(g2, g2))
    potential = -Z*(1/r1 + 1/r2) + 1/r12

    return kinetic + potential

### Approximating Ground-state Energy

In [5]:
# This function takes in a list of local energies,
# and outputs the sample average to approximate ground-state energy

def ground_state(local_energies):
    local_energies = np.array(local_energies)
    return np.mean(local_energies), np.var(local_energies)

### Optimizing the Trial Wavefunction

In [6]:
# This function scans over β and returns the one with minimum variance

def optimize(beta_grid, n_steps=30000):
    results = []
    best_beta = None
    best_var = np.inf

    for beta in beta_grid:
        samples = metropolis(beta, n_steps=n_steps)
        energies = [energy_local(R, beta) for R in samples]
        meanE, varE = ground_state(energies)

        results.append((beta, meanE, varE))
        if varE < best_var:
            best_var = varE
            best_beta = beta

    return best_beta, results

### Main Process

In [7]:
# This function represents the main process of one round of VMC

def main(beta, n_steps):
    samples = metropolis(beta, n_steps=n_steps)
    energies = [energy_local(R, beta) for R in samples]
    meanE, varE = ground_state(energies)
    return meanE, varE, energies

In [8]:
meanE, varE, energies = main(beta=0.3, n_steps=100)

print(f"Estimated ground-state energy: = {meanE:.6f} Hartree")
print(f"Variance of local energy: = {varE:.6f}")

Estimated ground-state energy: = -2.544229 Hartree
Variance of local energy: = 0.104850


### Iterations of the Process

In [9]:
#Optimizing beta parameter
beta_grid = np.linspace(0.0, 1.0, 6)
best_beta, results = optimize(beta_grid, n_steps=10)

meanE_bestbeta, varE_bestbeta, energies_bestbeta = main(best_beta, n_steps=100)
print(f"Estimated ground-state energy: = {meanE_bestbeta:.6f} Hartree")
print(f"Variance of local energy: = {varE_bestbeta:.6f}")


Estimated ground-state energy: = -2.816579 Hartree
Variance of local energy: = 0.016682


### Results

In [10]:
import pandas as pd

def summary_table_from_results(
    first_beta,
    first_mean,
    first_var,
    best_beta,
    opt_mean,
    opt_var
):
    exact_energy = -2.9037
    hf_energy = -2.8617

    df = pd.DataFrame({
        "Description": [
            "First iteration (initial β)",
            "Optimized β",
            "Exact helium ground-state energy",
            "Hartree–Fock energy"
        ],
        "β value": [
            first_beta,
            best_beta,
            "N/A",
            "N/A"
        ],
        "Energy (Hartree)": [
            first_mean,
            opt_mean,
            exact_energy,
            hf_energy
        ],
        "Variance": [
            first_var,
            opt_var,
            "N/A",
            "N/A"
        ]
    })
    
    return df
    
summary_table_from_results(
    first_beta = 0.3,
    first_mean = meanE,
    first_var = varE,
    best_beta = best_beta,
    opt_mean = meanE_bestbeta,
    opt_var = varE_bestbeta
)

,Description,β value,Energy (Hartree),Variance
0,First iteration (initial β),0.3,-2.544229,0.10485
1,Optimized β,0.4,-2.816579,0.016682
2,Exact helium ground-state energy,N/A,-2.903700,N/A
3,Hartree–Fock energy,N/A,-2.861700,N/A
